In [ ]:
!wget https://drive.google.com/file/d/1BRhlDz-JASvfEljf_XxrL-ZH4gUvOg0v/view

--2023-06-27 18:49:32--  https://drive.google.com/file/d/1BRhlDz-JASvfEljf_XxrL-ZH4gUvOg0v/view
Resolving drive.google.com (drive.google.com)... 108.177.120.139, 108.177.120.100, 108.177.120.102, ...
Connecting to drive.google.com (drive.google.com)|108.177.120.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘view’

view                    [ <=>                ]  77.87K  --.-KB/s    in 0.007s  

2023-06-27 18:49:33 (10.2 MB/s) - ‘view’ saved [79742]



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install fasttext

In [2]:
import nltk
import numpy as np
import pandas as pd
from tqdm import tqdm
from nltk import word_tokenize
import string
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import precision_recall_fscore_support, precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, recall_score
from sklearn.metrics import confusion_matrix
import fasttext
import os
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from torch.optim import Adam

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from time import time
from IPython.display import display

In [3]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
df = pd.read_csv('/content/drive/MyDrive/ML_NLP/data-train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/ML_NLP/pr-test-data.csv')

In [5]:
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [6]:
test_df.head()

,PhraseId,SentenceId,Phrase,ID
0,15061,8545,As exciting as all this exoticism might sound ...,1
1,15062,8545,As exciting as all this exoticism might sound ...,2
2,15063,8545,exciting as all this exoticism might sound to ...,3
3,15064,8545,exciting as all this exoticism might sound to ...,4
4,15065,8545,as all this exoticism might sound to the typic...,5


In [7]:
df.drop('PhraseId', axis=1, inplace=True)
test_df.drop('PhraseId', axis=1, inplace=True)

In [8]:
test_df.drop('ID', axis=1, inplace=True)

In [9]:
test_df.head()

,SentenceId,Phrase
0,8545,As exciting as all this exoticism might sound ...
1,8545,As exciting as all this exoticism might sound ...
2,8545,exciting as all this exoticism might sound to ...
3,8545,exciting as all this exoticism might sound to ...
4,8545,as all this exoticism might sound to the typic...


In [10]:
df.head()

,SentenceId,Phrase,Sentiment
0,1,A series of escapades demonstrating the adage ...,1
1,1,A series of escapades demonstrating the adage ...,2
2,1,A series,2
3,1,A,2
4,1,series,2


In [11]:
def preprocess_text(text, minimum_length=1, stopword_removal=False, stopwords_domain=[], lower_case=True,
                       punctuation_removal=True):

    normalized_tokens = word_tokenize(text)
    if stopword_removal:
        stopwords = [x.lower() for x in nltk.corpus.stopwords.words('english')]
        domain_stopwords = [x.lower() for x in stopwords_domain]
        normalized_tokens = [word for word in normalized_tokens if word.lower() not in domain_stopwords + stopwords]

    if punctuation_removal:
        normalized_tokens = [word for word in normalized_tokens if word not in string.punctuation]

    if lower_case:
        normalized_tokens = [word.lower() for word in normalized_tokens if len(word) > minimum_length]
    else:
        normalized_tokens = [word for word in normalized_tokens if len(word) > minimum_length]

    return normalized_tokens

In [12]:
df['Process Phrase'] = df['Phrase'].apply(lambda x: preprocess_text(x))
test_df['Process Phrase'] = test_df['Phrase'].apply(lambda x: preprocess_text(x))

In [13]:
df.head()

,SentenceId,Phrase,Sentiment,Process Phrase
0,1,A series of escapades demonstrating the adage ...,1,"[series, of, escapades, demonstrating, the, ad..."
1,1,A series of escapades demonstrating the adage ...,2,"[series, of, escapades, demonstrating, the, ad..."
2,1,A series,2,[series]
3,1,A,2,[]
4,1,series,2,[series]


In [14]:
test_df.head()

,SentenceId,Phrase,Process Phrase
0,8545,As exciting as all this exoticism might sound ...,"[as, exciting, as, all, this, exoticism, might..."
1,8545,As exciting as all this exoticism might sound ...,"[as, exciting, as, all, this, exoticism, might..."
2,8545,exciting as all this exoticism might sound to ...,"[exciting, as, all, this, exoticism, might, so..."
3,8545,exciting as all this exoticism might sound to ...,"[exciting, as, all, this, exoticism, might, so..."
4,8545,as all this exoticism might sound to the typic...,"[as, all, this, exoticism, might, sound, to, t..."


In [15]:
df.drop('Phrase', axis=1, inplace=True)
test_df.drop('Phrase', axis=1, inplace=True)

In [16]:
df = df[df['Process Phrase'].str.len() > 0]
test_df = test_df[test_df['Process Phrase'].str.len() > 0]

In [17]:
df.head()

,SentenceId,Sentiment,Process Phrase
0,1,1,"[series, of, escapades, demonstrating, the, ad..."
1,1,2,"[series, of, escapades, demonstrating, the, ad..."
2,1,2,[series]
4,1,2,[series]
5,1,2,"[of, escapades, demonstrating, the, adage, tha..."


In [18]:
test_df.head()

,SentenceId,Process Phrase
0,8545,"[as, exciting, as, all, this, exoticism, might..."
1,8545,"[as, exciting, as, all, this, exoticism, might..."
2,8545,"[exciting, as, all, this, exoticism, might, so..."
3,8545,"[exciting, as, all, this, exoticism, might, so..."
4,8545,"[as, all, this, exoticism, might, sound, to, t..."


In [19]:
df.drop('SentenceId', axis=1, inplace=True)
test_df.drop('SentenceId', axis=1, inplace=True)

<ipython-input-19-60d93badfdab>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('SentenceId', axis=1, inplace=True)


In [20]:
df.head(50)

,Sentiment,Process Phrase
0,1,"[series, of, escapades, demonstrating, the, ad..."
1,2,"[series, of, escapades, demonstrating, the, ad..."
2,2,[series]
4,2,[series]
5,2,"[of, escapades, demonstrating, the, adage, tha..."
6,2,[of]
7,2,"[escapades, demonstrating, the, adage, that, w..."
8,2,[escapades]
9,2,"[demonstrating, the, adage, that, what, is, go..."
10,2,"[demonstrating, the, adage]"


In [21]:
reviews = df['Process Phrase']
sentiment = df['Sentiment']

In [22]:
test_reviews = test_df['Process Phrase']

In [23]:
test_reviews

0       [as, exciting, as, all, this, exoticism, might...
1       [as, exciting, as, all, this, exoticism, might...
2       [exciting, as, all, this, exoticism, might, so...
3       [exciting, as, all, this, exoticism, might, so...
4       [as, all, this, exoticism, might, sound, to, t...
                              ...                        
1007       [struggles, of, the, working, class, to, life]
1008                  [of, the, working, class, to, life]
1009                      [the, working, class, to, life]
1010                                [the, working, class]
1011                                     [working, class]
Name: Process Phrase, Length: 1012, dtype: object

In [24]:
sentiment

0         1
1         2
2         2
4         2
5         2
         ..
155043    4
155044    2
155045    3
155046    4
155047    4
Name: Sentiment, Length: 154966, dtype: int64

In [25]:
type(sentiment)

pandas.core.series.Series

In [26]:
labels_df = sentiment.to_numpy()

In [27]:
labels_df

array([1, 2, 2, ..., 3, 4, 4])

In [28]:
# todo calculate the labels
enc = OneHotEncoder()
labels = enc.fit_transform(labels_df.reshape(-1, 1)).toarray()

In [29]:
labels

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])

In [30]:
class FastText:

    def __init__(self, preprocessor=None, method='skipgram'):
        self.method = method
        self.model = None
        self.preprocessor = preprocessor

    def train(self, texts):
        """
        train the fasttext model and save it into self.model
        Parameters
        ----------
        texts: list of list of str
        """
        with open('train.txt', 'w') as f:
            for text in texts:
                f.write(" ".join(text) + "\n")
        self.model = fasttext.train_unsupervised('train.txt', model=self.method, dim=100)

    def get_query_embedding(self, query, tf_idf_vectorizer):
        """
        get the embedding of the query. You can use the tf_idf_vectorizer to get the weights of the words in the query. preprocess the query using self.preprocessor if it is not None
        Parameters
        ----------
        query: str
        tf_idf_vectorizer: TfidfVectorizer
        Returns embedding of the query
        """

        query_embed = self.model.get_word_vector(query)
        return query_embed

    def save_FastText_model(self, path='FastText_model.bin'):
        self.model.save_model(path)

    def load_FastText_model(self, path="FastText_model.bin"):
        self.model = fasttext.load_model(path)

    def prepare(self, dataset, mode, save=False):
        if mode == 'train':
            self.train(dataset)
        if mode == 'load':
            self.load_FastText_model()
        if save:
            self.save_FastText_model()

In [31]:
FastText_model = FastText(preprocessor=preprocess_text)
FastText_model.prepare(reviews, mode='train', save=True)

In [32]:
embeddings = []
for row in reviews:
    embeddings.append(FastText_model.model.get_word_vector(' '.join(row)))
embeddings = np.array(embeddings)

In [33]:
FastText_model = FastText(preprocessor=preprocess_text)
FastText_model.prepare(test_reviews, mode='train', save=True)

In [34]:
test_embeddings = []
for row in test_reviews:
    test_embeddings.append(FastText_model.model.get_word_vector(' '.join(row)))
test_embeddings = np.array(test_embeddings)

In [35]:
test_embeddings

array([[-0.00077777,  0.00086581, -0.00063025, ...,  0.00088814,
        -0.00032133, -0.00041679],
       [-0.00099639,  0.00095429, -0.00067099, ...,  0.00093932,
        -0.00036976, -0.00046283],
       [-0.00106295,  0.00105452, -0.00075994, ...,  0.00104086,
        -0.00042679, -0.00048112],
       ...,
       [-0.00148535,  0.00189292, -0.00165462, ...,  0.00218079,
        -0.00080263, -0.00102927],
       [-0.0020382 ,  0.00267462, -0.00241234, ...,  0.00298689,
        -0.001014  , -0.00132466],
       [-0.00192319,  0.00244826, -0.0019968 , ...,  0.00241467,
        -0.00085988, -0.00106347]], dtype=float32)

In [36]:
test_embeddings.shape

(1012, 100)

In [37]:
embeddings.shape

(154966, 100)

In [38]:
labels.shape

(154966, 5)

In [39]:
X_train = embeddings
y_train = labels

In [40]:
from torch.utils.data import Dataset

class PapersDataSet(Dataset):
    def __init__(self, embeddings: list, labels: list):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, i):
        return self.embeddings[i], self.labels[i]

In [41]:
train_dataset = PapersDataSet(X_train, y_train)

batch_size  = 64

train_loader = DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [42]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [60]:
import torch
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self, in_features=1000, num_classes=5):
        super().__init__()
        self.seq = nn.Sequential(
            nn.BatchNorm1d(in_features),
            nn.Linear(in_features, 512),  # Additional linear layer
            nn.ReLU(),                     # Activation function
            nn.Linear(512, 256),            # Additional linear layer
            nn.ReLU(),                     # Activation function
            nn.Linear(256, num_classes),    # Final linear layer
            nn.Softmax(1),
        )

    def forward(self, x):
        return self.seq(x)

# Define the number of classes
num_classes = 5

# Create an instance of the ResNet-50 model
model = CNN(100,5)


In [61]:
model = model.to(device)

In [62]:

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [63]:
def eval_epoch(model: nn.Module, criterion: nn.Module, dataloader: torch.utils.data.DataLoader, test_mode=False):
    """
    Evaluate the model on the given dataloader. used for validation and test
    Parameters
    ----------
    model: nn.Module
    criterion: nn.Module
    dataloader: torch.utils.data.DataLoader
    test_mode: bool
        If True, the function will print 'Test' instead of 'Validation'
    Returns
    -------
    eval_loss: float
        The loss on the given dataloader
    predicted_labels: list
        The predicted labels
    true_labels: list
        The true labels
    f1_score_macro: float
        The f1 score on the given dataloader
    """
    eval_loss = 0
    predicted_labels = []
    true_labels = []
    with torch.no_grad():
        for i, (x, labels) in enumerate(dataloader):
            x, labels = x.to(device, dtype=torch.float), labels.to(device, dtype=torch.float)
            outputs = model(x)
            loss = criterion(outputs, labels)
            eval_loss += loss.item()
            predicted_targets = outputs.argmax(dim=1)
            truths = labels.argmax(dim=1)
            predicted_labels.extend(predicted_targets.detach().cpu())
            true_labels.extend(truths.detach().cpu())

    f1_score_macro = f1_score(true_labels, predicted_labels, average='macro')


    return eval_loss, predicted_labels, true_labels, f1_score_macro

In [65]:
"""
Train the model for num_epochs epochs
epoch_true and epoch_all are used to calculate the accuracy.
epoch_true is the number of correct predictions and epoch_all is the total number of predictions in the epoch
"""
num_epochs = 25

train_loss_arr, val_loss_arr = [], []
f1_macro_scores = []
f1_train_scores = []
for epoch in range(num_epochs):
    start_time = time()

    train_loss, val_loss = 0, 0
    epoch_all = 0
    epoch_true = 0
    predicted_labels = []
    true_labels = []

    model.train()

    for i, (x, labels) in enumerate(train_loader):
        x, labels = x.to(device, dtype=torch.float), labels.to(device, dtype=torch.float)
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        predicted_targets = outputs.argmax(dim=1)
        truths = labels.argmax(dim=1)
        predicted_labels.extend(predicted_targets.detach().cpu())
        true_labels.extend(truths.detach().cpu())
        epoch_true += (predicted_targets == truths).sum().item()
        epoch_all += len(predicted_targets)

    f1_train_scores.append(f1_score(true_labels, predicted_labels, average='macro'))
    end_time = time()

    print(f'Epoch {epoch + 1} finished in {end_time - start_time:.2f}s')
    print(f"[Epoch {epoch + 1}]\t"
        f"Train Loss: {train_loss:.4f}\t")

Epoch 1 finished in 22.38s
[Epoch 1]	Train Loss: 3188.8677	
Epoch 2 finished in 19.66s
[Epoch 2]	Train Loss: 3183.1891	
Epoch 3 finished in 20.23s
[Epoch 3]	Train Loss: 3172.4241	
Epoch 4 finished in 20.05s
[Epoch 4]	Train Loss: 3166.3229	
Epoch 5 finished in 20.34s
[Epoch 5]	Train Loss: 3160.6468	
Epoch 6 finished in 20.06s
[Epoch 6]	Train Loss: 3152.6647	
Epoch 7 finished in 19.98s
[Epoch 7]	Train Loss: 3146.1837	
Epoch 8 finished in 20.11s
[Epoch 8]	Train Loss: 3140.9544	
Epoch 9 finished in 20.40s
[Epoch 9]	Train Loss: 3138.8516	
Epoch 10 finished in 20.07s
[Epoch 10]	Train Loss: 3134.4325	
Epoch 11 finished in 21.83s
[Epoch 11]	Train Loss: 3125.3205	
Epoch 12 finished in 19.87s
[Epoch 12]	Train Loss: 3124.0954	
Epoch 13 finished in 19.80s
[Epoch 13]	Train Loss: 3119.9392	
Epoch 14 finished in 20.21s
[Epoch 14]	Train Loss: 3115.6144	
Epoch 15 finished in 19.99s
[Epoch 15]	Train Loss: 3113.0946	
Epoch 16 finished in 19.97s
[Epoch 16]	Train Loss: 3107.2601	
Epoch 17 finished in 20.90

In [70]:
test_tensor = torch.tensor(test_embeddings)
test_tensor = test_tensor.to(device)

# Set the model to evaluation mode
model.eval()

# Pass the test samples through the model to get the outputs
with torch.no_grad():
    outputs = model(test_tensor)
outputs = outputs.cpu()
# Convert the outputs to a numpy array
output_array = outputs.numpy()

# Print the output array
print(output_array)

[[0.0000000e+00 1.7386288e-15 1.0000000e+00 2.0282278e-12 0.0000000e+00]
 [0.0000000e+00 1.8957925e-15 1.0000000e+00 2.1349708e-12 0.0000000e+00]
 [0.0000000e+00 2.0184611e-15 1.0000000e+00 2.1879319e-12 0.0000000e+00]
 ...
 [0.0000000e+00 3.5895901e-15 1.0000000e+00 2.8818354e-12 0.0000000e+00]
 [0.0000000e+00 5.9219144e-15 1.0000000e+00 3.6728142e-12 0.0000000e+00]
 [0.0000000e+00 4.7944322e-15 1.0000000e+00 3.3819679e-12 0.0000000e+00]]


In [71]:
output_array[10]

array([0.0000000e+00, 2.3287712e-15, 1.0000000e+00, 2.2748678e-12,
       0.0000000e+00], dtype=float32)

In [72]:
indices = np.argmax(output_array, axis=1)

# Create a new array with the same length as output_array, filled with zeros
result_array = np.zeros(len(output_array))

# Iterate over each row in output_array
for i in range(output_array.shape[0]):
    # Get the index of the maximum value for the current row
    row_indices = np.argmax(output_array[i])

    # Set the index in the current row to 1
    result_array[i] = row_indices

# Print the result_array
print(result_array[10])

2.0


In [73]:
result_array

array([2., 2., 2., ..., 2., 2., 2.])

In [74]:
ids = np.arange(1, len(result_array) + 1)

# Create a DataFrame with the result_array and ids
df = pd.DataFrame({'Sentiment': result_array, 'ID': ids})

# Print the DataFrame
print(df)

      Sentiment    ID
0           2.0     1
1           2.0     2
2           2.0     3
3           2.0     4
4           2.0     5
...         ...   ...
1007        2.0  1008
1008        2.0  1009
1009        2.0  1010
1010        2.0  1011
1011        2.0  1012

[1012 rows x 2 columns]


In [75]:
df.to_csv('output_nn.csv', index=False)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report


classifier = LinearSVC()
classifier.fit(X_train, labels_df)

# Make predictions on the test set
y_pred = classifier.predict(test_embeddings)

# Evaluate the model


In [ ]:
y_pred

array([2, 2, 2, ..., 2, 2, 2])

In [ ]:
ids = np.arange(1, len(y_pred) + 1)

# Create a DataFrame with the result_array and ids
df = pd.DataFrame({'Sentiment': y_pred, 'ID': ids})

# Print the DataFrame
print(df)

      Sentiment    ID
0             2     1
1             2     2
2             2     3
3             2     4
4             2     5
...         ...   ...
1007          2  1008
1008          2  1009
1009          2  1010
1010          2  1011
1011          2  1012

[1012 rows x 2 columns]


In [ ]:
df.to_csv('output_svm.csv', index=False)

SVM not using proccess

In [59]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report


# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/ML_NLP/data-train.csv')
data_test = pd.read_csv('/content/drive/MyDrive/ML_NLP/pr-test-data.csv')

# Split the data into input features (reviews) and target variable (sentiment)
reviews = data['Phrase']
sentiment = data['Sentiment']

# Split the dataset into training and testing sets
X_train2 = reviews
X_test2 = data_test['Phrase']
y_train2 = sentiment


vectorizer = TfidfVectorizer(max_features=100)
X_train_vectorized = vectorizer.fit_transform(X_train2)
X_test_vectorized = vectorizer.transform(X_test2)

# Train a linear support vector classifier (SVC)
classifier = LinearSVC()
classifier.fit(X_train_vectorized, y_train2)

# Make predictions on the test set
y_pred = classifier.predict(X_test_vectorized)


In [60]:
y_pred

array([3, 2, 2, ..., 2, 2, 2])

In [61]:
ids = np.arange(1, len(y_pred) + 1)

# Create a DataFrame with the result_array and ids
df = pd.DataFrame({'Sentiment': y_pred, 'ID': ids})

# Print the DataFrame
print(df)

      Sentiment    ID
0             3     1
1             2     2
2             2     3
3             2     4
4             2     5
...         ...   ...
1007          2  1008
1008          2  1009
1009          2  1010
1010          2  1011
1011          2  1012

[1012 rows x 2 columns]


In [62]:
df.to_csv('output_svm_NP.csv', index=False)

using *MultinomialNB*

In [50]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler

In [51]:
#training the model
mnb=MultinomialNB()
#fitting the svm for bag of words
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_train)
mnb_model = mnb.fit(X_scaled, labels_df)
print(mnb_model)
#fitting the svm for tfidf features
y_pred = mnb_model.predict(test_embeddings)

MultinomialNB()


In [52]:
ids = np.arange(1, len(y_pred) + 1)

# Create a DataFrame with the result_array and ids
df = pd.DataFrame({'Sentiment': y_pred, 'ID': ids})

# Print the DataFrame
print(df)

      Sentiment    ID
0             2     1
1             2     2
2             2     3
3             2     4
4             2     5
...         ...   ...
1007          2  1008
1008          2  1009
1009          2  1010
1010          2  1011
1011          2  1012

[1012 rows x 2 columns]


In [53]:
df.to_csv('output_NB.csv', index=False)